In [17]:
from elasticsearch import Elasticsearch

from elasticsearch.helpers import bulk
es = Elasticsearch([{"host": "localhost", "port": 9200, "scheme": "http"}])
print(es.ping())

True


In [14]:
import gspread


gs = gspread.service_account(filename='service_account.json')
sh_clean = gs.open_by_key('1yuCSKE7hbbX_ChOu9vCFJVdnAkC3T9rSg9pFpZIulOA')
worksheet_clean = sh_clean.sheet1
clean_data = worksheet_clean.get_all_values()

import pandas as pd

df_graph = pd.DataFrame(clean_data[1:], columns=clean_data[0])

df_graph['x'] = df_graph['x'].str.replace(',', '.')
df_graph['y'] = df_graph['y'].str.replace(',', '.')
df_graph['x'] = df_graph['x'].astype(float)
df_graph['y'] = df_graph['y'].astype(float)


In [16]:
print(df_graph.columns)

Index(['area', 'cluster', 'cluster_name', 'keyword', 'count', 'x', 'y',
       'color'],
      dtype='object')


In [30]:
data_to_write = df_graph.to_dict(orient='records')
index_name = 'hse_test_task'

field_mappings = {
    "area": {"type": "text"},
    "cluster": {"type": "text"},
    "cluster_name": {"type": "text"},
    "keyword": {"type": "keyword"},
    "count": {"type": "integer"},
    "x": {"type": "float"},
    "y": {"type": "float"},
    "color": {"type": "text"}
}


index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 1
    }
}

es.indices.create(index=index_name, ignore=400)


/var/folders/z5/wq_n2wnn1d39t43rd8s7f6g00000gn/T/ipykernel_1370/2184878193.py:23: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index=index_name, ignore=400)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'hse_test_task'})

In [31]:
es.indices.put_mapping(index=index_name, body={"properties": field_mappings})

/var/folders/z5/wq_n2wnn1d39t43rd8s7f6g00000gn/T/ipykernel_1370/3590665393.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.put_mapping(index=index_name, body={"properties": field_mappings})


ObjectApiResponse({'acknowledged': True})

In [32]:
actions = [
    {
        "_index": index_name,
        "_source": data
    }
    for data in data_to_write
]

success, failed = bulk(es, actions, index=index_name, raise_on_error=True)


In [34]:
search_result = es.search(index=index_name, body={
    "size": 10,
    "query": {
        "match_all": {}
    }
})

# Получите результаты запроса
hits = search_result['hits']['hits']

# Выведите полученные документы
for hit in hits:
    print(hit['_source'])

{'area': 'ar\\vr', 'cluster': '0', 'cluster_name': 'Кластер 0', 'keyword': 'written conflict fabulous', 'count': '1443', 'x': 2.991167387, 'y': 7.106798807, 'color': '#ffd94a'}
{'area': 'ar\\vr', 'cluster': '0', 'cluster_name': 'Кластер 0', 'keyword': 'reservations linking', 'count': '751', 'x': 10.19560242, 'y': 12.25949571, 'color': '#ffd94a'}
{'area': 'ar\\vr', 'cluster': '0', 'cluster_name': 'Кластер 0', 'keyword': 'interfaces neutral', 'count': '586', 'x': 10.44353325, 'y': 13.80991497, 'color': '#ffd94a'}
{'area': 'ar\\vr', 'cluster': '0', 'cluster_name': 'Кластер 0', 'keyword': 'committees parallel', 'count': '173', 'x': 6.735259815, 'y': 3.613982972, 'color': '#ffd94a'}
{'area': 'ar\\vr', 'cluster': '1', 'cluster_name': 'Кластер 1', 'keyword': 'postcards looked republic detector', 'count': '1397', 'x': 10.47447422, 'y': 6.220011967, 'color': '#3cb7cc'}
{'area': 'ar\\vr', 'cluster': '1', 'cluster_name': 'Кластер 1', 'keyword': 'michael tobacco', 'count': '1007', 'x': 6.21087987,

/var/folders/z5/wq_n2wnn1d39t43rd8s7f6g00000gn/T/ipykernel_1370/1285631464.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  search_result = es.search(index=index_name, body={


In [41]:
res = es.get(index=index_name, id='OdKPrIoBgai6cQ8b6JuY')

In [42]:
print(res)

{'_index': 'hse_test_task', '_id': 'OdKPrIoBgai6cQ8b6JuY', '_version': 1, '_seq_no': 6, '_primary_term': 1, 'found': True, '_source': {'area': 'ar\\vr', 'cluster': '1', 'cluster_name': 'Кластер 1', 'keyword': 'filling volunteers academics', 'count': '773', 'x': 10.83391996, 'y': 8.652736658, 'color': '#3cb7cc'}}
